In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz

import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
# ignore warnings
import warnings
warnings.filterwarnings("ignore")
""
import acquire
import prepares

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 0 to 149
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   measurement_id  150 non-null    int64  
 1   sepal_length    150 non-null    float64
 2   sepal_width     150 non-null    float64
 3   petal_length    150 non-null    float64
 4   petal_width     150 non-null    float64
 5   species_id      150 non-null    int64  
 6   species_id.1    150 non-null    int64  
 7   species_name    150 non-null    object 
dtypes: float64(4), int64(3), object(1)
memory usage: 10.5+ KB
measurement_id
sepal_length
sepal_width
petal_length
petal_width
species_id
species_id.1
species_name
<class 'pandas.core.frame.DataFrame'>
Int64Index: 889 entries, 0 to 890
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    889 non-null    int64  
 1   passenger_id  889 non-null    int64  
 2  

<h1><h1><h1 style= 'color: darkcyan'>Model Exercise</h1>

In this exercise, we'll continue working with the titanic dataset and building logistic regression models. Throughout this exercise, be sure you are training, evaluation, and comparing models on the train and validate datasets. The test dataset should only be used for your final model.

For all of the models you create, choose a threshold that optimizes for accuracy.

Do your work for these exercises in either a notebook or a python script named model within your classification-exercises repository. Add, commit, and push your work.
    
<ol>




<li>Try out other combinations of features and models. </li>


<li>Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train? </li>

<li>Bonus1 How do different strategies for handling the missing values in the age column affect model performance?</li>

<li>Bonus2: How do different strategies for encoding sex affect model performance?</li>

<li>Bonus3: scikit-learn's LogisticRegression classifier is actually applying a regularization penalty to the coefficients by default. This penalty causes the magnitude of the coefficients in the resulting model to be smaller than they otherwise would be. This value can be modified with the C hyper parameter. Small values of C correspond to a larger penalty, and large values of C correspond to a smaller penalty.
    
Try out the following values for C and note how the coefficients and the model's performance on both the dataset it was trained on and on the validate split are affected.<br>
C=.01,.1,1,10,100,1000 </li>
<li> Bonus Bonus: how does scaling the data interact with your choice of C? </li>
    </ol>

In [3]:

train, validate, test = prepares.prep_titanic()


In [4]:
demo_train = train[['age', 'survived']]
demo_train2 = train[['age', 'pclass', 'survived']]

X_train = demo_train[['age']]
X_train = demo_train.drop(columns=['survived'])

X_train2 = train[['age', 'pclass']]
X_train2 = demo_train.drop(columns=['survived'])
y_demo_train = demo_train[['survived']]
y_demo_train = demo_train['survived']
y_demo_train = demo_train.survived

In [5]:
train.survived.value_counts(normalize=True)

0    0.617706
1    0.382294
Name: survived, dtype: float64

In [6]:
logit1 = LogisticRegression()

X_train1 = train.drop(columns=['survived', 'embark_town', 'class', 'embarked', 'sex', 'Unnamed: 0'])
y_train = train.survived

logit1 = logit1.fit(X_train1, y_train)
print(logit1.coef_)
print(X_train1.columns)

[[ 9.42277253e-04 -2.88304459e-01 -1.03271631e-03 -2.88983465e-01
  -2.34954082e-01  1.18453069e-02 -4.24606131e-01  2.97052562e-01
   3.07306293e-01 -2.25787328e+00]]
Index(['passenger_id', 'pclass', 'age', 'sibsp', 'parch', 'fare', 'alone', 'Q',
       'S', 'is_male'],
      dtype='object')


In [7]:
y_pred1 = logit1.predict(X_train1)

In [8]:
logit1.score(X_train1, y_train)

0.7927565392354124

In [9]:
X_train2 = train[['age', 'fare', 'pclass']]
y_train = train.survived

logit2 = LogisticRegression()
logit2 = logit2.fit(X_train2, y_train)

print(logit2.coef_)
print(X_train2.columns)

[[-0.03051881  0.00266519 -0.97983178]]
Index(['age', 'fare', 'pclass'], dtype='object')


In [10]:
logit2.score(X_train2, y_train)

0.716297786720322

<li>Include sex in your model as well. Note that you'll need to encode or create a dummy variable of this feature before including it in a model. </li>

In [12]:
X_train3 = train[['age', 'fare', 'pclass', 'is_male']]
y_train = train.survived

logit3 = LogisticRegression()
logit3 = logit3.fit(X_train3, y_train)

print(logit3.coef_)
print(X_train3.columns)
print(logit3.score(X_train3, y_train))

[[-2.66594879e-02  9.02716903e-04 -1.11402368e+00 -2.45878213e+00]]
Index(['age', 'fare', 'pclass', 'is_male'], dtype='object')
0.7987927565392354


In [13]:
# SIDE NOTE
# demonstrating the weights with different values
# a weight of .003 with a $500 fare is not much different 
# than a weight of 2.3 with a 1 for sex_male

500*.003 
1*2.3

2.3

In [15]:
X_train4 = train[['is_male']]
y_train = train.survived

def my_logit(X_train):
    my_logit = LogisticRegression()
    my_logit = my_logit.fit(X_train, y_train)
    return my_logit, my_logit.coef_, my_logit.score(X_train, y_train)

In [16]:
logit4, coefs, accuracy = my_logit(X_train4)
print(coefs, accuracy)


[[-2.37681345]] 0.7847082494969819


<li>Try out other combinations of features and models. </li>


In [17]:
train.columns

Index(['Unnamed: 0', 'passenger_id', 'survived', 'pclass', 'sex', 'age',
       'sibsp', 'parch', 'fare', 'embarked', 'class', 'embark_town', 'alone',
       'Q', 'S', 'is_male'],
      dtype='object')

In [19]:
X_train5 = train[['sibsp', 'parch', 'alone', 'Q', 'S']]

logit5, coefs, accuracy = my_logit(X_train5)
print(coefs, accuracy)

[[-0.58064188 -0.08586553 -1.81001298  0.11658434 -0.35790445]] 0.6941649899396378


In [24]:
X_train6 = train[['alone', 'is_male', 'pclass']]

logit6, coefs, accuracy = my_logit(X_train6)
print(coefs, accuracy)

[[-0.30828946 -2.40744024 -0.95701015]] 0.7847082494969819


<li> Use you best 3 models to predict and evaluate on your validate sample. </li>

In [29]:
X_validate1 = validate.drop(columns=['survived', 'embark_town', 'class', 'embarked', 'sex', 'Unnamed: 0'])
X_validate4 = validate[['is_male']]
X_validate6 = validate[['alone', 'pclass', 'is_male']]

y_validate = validate.survived

acc1 = logit1.score(X_validate1, y_validate)
acc4 = logit4.score(X_validate4, y_validate)
acc6 = logit6.score(X_validate6, y_validate)

print(acc1, acc4, acc6)

0.780373831775701 0.7663551401869159 0.7429906542056075


<li> Use you best 3 models to predict and evaluate on your validate sample. </li>

<li>Choose you best model from the validation performation, and evaluate it on the test dataset. How do the performance metrics compare to validate? to train?</li>

In [46]:
y1_pred = logit1.predict(X_validate1)
y4_pred = logit4.predict(X_validate4)
y6_pred = logit6.predict(X_validate6)

print("y1: All Vars:\n", classification_report(y_validate, y1_pred))

print("y4: Sex_male only:\n", classification_report(y_validate, y4_pred))

print("y6: Age, Pclass, Sex_male:\n", classification_report(y_validate, y6_pred))

y1: All Vars:
               precision    recall  f1-score   support

           0       0.80      0.86      0.83       132
           1       0.74      0.66      0.70        82

    accuracy                           0.78       214
   macro avg       0.77      0.76      0.76       214
weighted avg       0.78      0.78      0.78       214

y4: Sex_male only:
               precision    recall  f1-score   support

           0       0.80      0.83      0.81       132
           1       0.71      0.67      0.69        82

    accuracy                           0.77       214
   macro avg       0.75      0.75      0.75       214
weighted avg       0.76      0.77      0.77       214

y6: Age, Pclass, Sex_male:
               precision    recall  f1-score   support

           0       0.71      1.00      0.83       132
           1       1.00      0.33      0.50        82

    accuracy                           0.74       214
   macro avg       0.85      0.66      0.66       214
weighted av

<h1><h1 style= 'color: darkcyan'> Decision Tree Exercises </h1>
In this exercise, we'll continue working with the titanic dataset and building logistic regression models. Throughout this exercise, be sure you are training, evaluation, and comparing models on the train and validate datasets. The test dataset should only be used for your final model.

Continue working in your model file. Add, commit, and push your changes.
<ol>
    <li>Fit the decision tree classifier to your training sample and transform (i.e. make predictions on the training sample)</li>
<li>Evaluate your in-sample results using the model score, confusion matrix, and classification report. </li>
<li>Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.</li>
<li>Run through steps 2-4 using a different max_depth value. </li>
<li>Which performs better on your in-sample data? </li>

In [4]:
titanic = sns.load_dataset("titanic")
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [6]:
## drop columns
titanic = titanic[["survived", "pclass", "sex", "age", "fare"]]
titanic.head()

,survived,pclass,sex,age,fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500


In [7]:
## making the sex of titanic into a binary integer
def gender(gender):
    if gender == "male":
        return 0
    else:
        return 1

In [9]:
## applying the function to the dataframe
titanic.sex = titanic.sex.apply(gender)
titanic.head()

,survived,pclass,sex,age,fare
0,0,3,0,22.0,7.2500
1,1,1,1,38.0,71.2833
2,1,3,1,26.0,7.9250
3,1,1,1,35.0,53.1000
4,0,3,0,35.0,8.0500


In [10]:
## showing the nulls
print(f"Survived nulls: {df.survived.isna().sum()}")
print(f"Class nulls:  {df.pclass.isna().sum()}")
print(f"Gender nulls: {df.sex.isna().sum()}")
print(f"Age nulls: {df.age.isna().sum()}")
print(f"Fare nulls: {df.fare.isna().sum()}")

Survived nulls: 0
Class nulls:  0
Gender nulls: 0
Age nulls: 177
Fare nulls: 0


In [11]:
titanic.isna().sum()

survived      0
pclass        0
sex           0
age         177
fare          0
dtype: int64

In [55]:
# getting the median age while ignoring nulls
median_age = df[df.age.notnull()].age.median()
median_age

28.0

In [57]:
# dropping the nulls by filling the nulls w/ the median 
df.age = df.age.fillna(median_age)
df.age
#print(f"Age nulls: {df.age.isna().sum()}")

0      22.0
1      38.0
2      26.0
3      35.0
4      35.0
       ... 
886    27.0
887    19.0
888    28.0
889    26.0
890    32.0
Name: age, Length: 891, dtype: float64

In [58]:
# the pandas .median method ignores nulls
df.age.median()

28.0

In [59]:
# Setup the X and y variables
X = titanic.drop("survived", axis=1)
y = titanic[["survived"]]
X.head()

,pclass,sex,age,fare
0,3,0,22.0,7.2500
1,1,1,38.0,71.2833
2,3,1,26.0,7.9250
3,1,1,35.0,53.1000
4,3,0,35.0,8.0500


In [60]:
y.head()

,survived
0,0
1,1
2,1
3,1
4,0


In [61]:
# Train/Test/Split set up

# train_test_split gives us 2 datasets
X_train_validate, X_test, y_train_validate, y_test = train_test_split(X, y, test_size = .30, random_state = 123, stratify=y.survived)

In [62]:
# Now, let's split our "train/validate into train and validate"
X_train, X_validate, y_train, y_validate = train_test_split(X_train_validate, y_train_validate, test_size=0.3, random_state = 123, stratify=y_train_validate)

In [63]:
# classfying.
clf = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=123)

In [73]:
# My X train wouldn't fit because it still had naans. I found the naans and then filled them with the mean.
np.where(np.isnan(X_train))
X_train=X_train.fillna(X_train.mean())

In [74]:
# Fitting the x train and the y train
clf.fit(X_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=5, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=123, splitter='best')

In [75]:
## Predicting Y based off the fitted x train
y_pred = clf.predict(X_train)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0,

<h1><h1 style= 'color: darkcyan'> Random Forest Exercises </h1>
Continue working in your model file. Be sure to add, commit, and push your changes.
<ol>
    <li>Fit the Random Forest classifier to your training sample and transform (i.e. make predictions on the training sample) setting the random_state accordingly and setting min_samples_leaf = 1 and max_depth = 20. </li>
<li>Evaluate your results using the model score, confusion matrix, and classification report.</li>
<li>Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.</li>
<li>Run through steps increasing your min_samples_leaf to 5 and decreasing your max_depth to 3.</li>
<li>What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?</li>
<li>After making a few models, which one has the best performance (or closest metrics) on both train and validate?</li>

<h1><h1 style= 'color: darkcyan'> KNN Exercises </h1>
Continue working in your model notebook or python script.
<ol>
    <li>Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)</li>
<li>Evaluate your results using the model score, confusion matrix, and classification report.</li>
<li>Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.</li>
<li>Run through steps 2-4 setting k to 10</li>
<li>Run through setps 2-4 setting k to 20</li>
<li>What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?</li>

In [76]:
! git add "model.ipynb"

In [ ]:
! GIT CO